In [1]:
# Bibliotecas a utilizar
import pandas as pd
import gzip
import ast


In [2]:
# Se guarda el path del json en una variable
ui_path = '../data/raw/users_items.json.gz'

In [3]:
def descompimir_json(ruta, variable_anidada):
    '''Función que recibe una ruta de acceso a un archivo json anidado y carga la información en un
    DataFrame de Pandas'''
    fila = []
    with gzip.open(ruta, 'rt', encoding='MacRoman') as archivo:
      for line in archivo.readlines():
          fila.append(ast.literal_eval(line))

    df = pd.DataFrame(fila)                                                 
    df = df.explode(variable_anidada).reset_index()                         
    df = df.drop(columns="index")                                           
    df = pd.concat([df, pd.json_normalize(df[variable_anidada])], axis=1)   
    df = df.drop(columns=variable_anidada)                                  

    return df

In [4]:
# Aplicamos la función para descomprimir el json
df_users_items = descompimir_json(ui_path, 'items')

In [5]:
# Revisar los primeros 5 registros
df_users_items.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0


In [6]:
# Empecemos por ver que tipo de datos tenemos
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 315.6+ MB


In [7]:
# Recorremos con un for el dataframe para ver el tipo de dato de cada columna
for c in df_users_items:
    print(c, df_users_items[c].apply(lambda x: type(x)).value_counts())

user_id user_id
<class 'str'>    5170015
Name: count, dtype: int64
items_count items_count
<class 'int'>    5170015
Name: count, dtype: int64
steam_id steam_id
<class 'str'>    5170015
Name: count, dtype: int64
user_url user_url
<class 'str'>    5170015
Name: count, dtype: int64
item_id item_id
<class 'str'>      5153209
<class 'float'>      16806
Name: count, dtype: int64
item_name item_name
<class 'str'>      5153209
<class 'float'>      16806
Name: count, dtype: int64
playtime_forever playtime_forever
<class 'float'>    5170015
Name: count, dtype: int64
playtime_2weeks playtime_2weeks
<class 'float'>    5170015
Name: count, dtype: int64


In [8]:
# Revisamos los valores nulos
df_users_items.isna().sum()

user_id                 0
items_count             0
steam_id                0
user_url                0
item_id             16806
item_name           16806
playtime_forever    16806
playtime_2weeks     16806
dtype: int64

In [9]:
# Veamos los valores flotantes de la columna 'item_id'
df_users_items[df_users_items['item_id'].apply(lambda x: type(x) == float)]

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
3733,Wackky,0,76561198039117046,http://steamcommunity.com/id/Wackky,NaN,NaN,NaN,NaN
3849,76561198079601835,0,76561198079601835,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN
6019,hellom8o,0,76561198117222320,http://steamcommunity.com/id/hellom8o,NaN,NaN,NaN,NaN
6523,starkillershadow553,0,76561198059648579,http://steamcommunity.com/id/starkillershadow553,NaN,NaN,NaN,NaN
7237,darkenkane,0,76561198058876001,http://steamcommunity.com/id/darkenkane,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5169470,76561198316380182,0,76561198316380182,http://steamcommunity.com/profiles/76561198316...,NaN,NaN,NaN,NaN
5169471,76561198316970597,0,76561198316970597,http://steamcommunity.com/profiles/76561198316...,NaN,NaN,NaN,NaN
5169472,76561198318100691,0,76561198318100691,http://steamcommunity.com/profiles/76561198318...,NaN,NaN,NaN,NaN
5170006,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,NaN,NaN,NaN,NaN


In [10]:
# Porcentaje de valores nulos en el dataframe por columna
round(df_users_items.isna().sum() / len(df_users_items) * 100, 2)

user_id             0.00
items_count         0.00
steam_id            0.00
user_url            0.00
item_id             0.33
item_name           0.33
playtime_forever    0.33
playtime_2weeks     0.33
dtype: float64

In [11]:
# Eliminamos los valores nulos al ser un porcentaje muy bajo
df_users_items = df_users_items.dropna()

In [12]:
df_users_items

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...
5170009,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,346330,BrainBread 2,0.0,0.0
5170010,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,373330,All Is Dust,0.0,0.0
5170011,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570,You Have 10 Seconds 2,4.0,4.0


In [13]:
# Eliminamos columnas que no vamos a utilizar
df_users_items.drop(['items_count','steam_id','user_url','playtime_2weeks'], axis=1, inplace=True)

In [14]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 5170013
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           object 
 2   item_name         object 
 3   playtime_forever  float64
dtypes: float64(1), object(3)
memory usage: 196.6+ MB


In [15]:
# Recorremos con un for el dataframe para ver el tipo de dato de cada columna
for c in df_users_items:
    print(c, df_users_items[c].apply(lambda x: type(x)).value_counts())

user_id user_id
<class 'str'>    5153209
Name: count, dtype: int64
item_id item_id
<class 'str'>    5153209
Name: count, dtype: int64
item_name item_name
<class 'str'>    5153209
Name: count, dtype: int64
playtime_forever playtime_forever
<class 'float'>    5153209
Name: count, dtype: int64


In [16]:
df_users_items.describe()

,playtime_forever
count,5.153209e+06
mean,9.914951e+02
std,5.418204e+03
min,0.000000e+00
25%,0.000000e+00
50%,3.400000e+01
75%,3.550000e+02
max,6.427730e+05


In [17]:
# Revisar si hay valores iguales o menores a 0 en la columna 'playtime_forever'
df_users_items[df_users_items['playtime_forever'] <= 0]

,user_id,item_id,item_name,playtime_forever
1,76561197970982479,20,Team Fortress Classic,0.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0
5,76561197970982479,60,Ricochet,0.0
6,76561197970982479,70,Half-Life,0.0
...,...,...,...,...
5170003,76561198326700687,519170,The Journey Home,0.0
5170004,76561198326700687,358390,Stargunner,0.0
5170005,76561198326700687,521570,You Have 10 Seconds 2,0.0
5170009,76561198329548331,346330,BrainBread 2,0.0


In [18]:
# Ahora revisar si hay valores menores a 0 en la columna 'playtime_forever'
df_users_items[df_users_items['playtime_forever'] == 0]

,user_id,item_id,item_name,playtime_forever
1,76561197970982479,20,Team Fortress Classic,0.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0
5,76561197970982479,60,Ricochet,0.0
6,76561197970982479,70,Half-Life,0.0
...,...,...,...,...
5170003,76561198326700687,519170,The Journey Home,0.0
5170004,76561198326700687,358390,Stargunner,0.0
5170005,76561198326700687,521570,You Have 10 Seconds 2,0.0
5170009,76561198329548331,346330,BrainBread 2,0.0


In [19]:
# Considerando que más adelante vamos a utilizar la columna 'playtime_forever' para identificar tiempo de juego, los valores 0 no nos sirven, por lo que los eliminamos
df_users_items = df_users_items[df_users_items['playtime_forever'] > 0]

In [20]:
df_users_items

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
2,76561197970982479,30,Day of Defeat,7.0
8,76561197970982479,300,Day of Defeat: Source,4733.0
9,76561197970982479,240,Counter-Strike: Source,1853.0
10,76561197970982479,3830,Psychonauts,333.0
...,...,...,...,...
5170007,76561198329548331,304930,Unturned,677.0
5170008,76561198329548331,227940,Heroes & Generals,43.0
5170011,76561198329548331,388490,One Way To Die: Steam Edition,3.0
5170012,76561198329548331,521570,You Have 10 Seconds 2,4.0


In [21]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3285246 entries, 0 to 5170013
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           object 
 2   item_name         object 
 3   playtime_forever  float64
dtypes: float64(1), object(3)
memory usage: 125.3+ MB


In [22]:
# Cambiar tipo de dato de la columna 'item_id' a int
df_users_items['item_id'] = df_users_items['item_id'].astype(int)

C:\Users\jacev\AppData\Local\Temp\ipykernel_14360\3847867734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_users_items['item_id'] = df_users_items['item_id'].astype(int)


In [23]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3285246 entries, 0 to 5170013
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           int32  
 2   item_name         object 
 3   playtime_forever  float64
dtypes: float64(1), int32(1), object(2)
memory usage: 112.8+ MB


In [24]:
# Usando la columna 'playtime_forever' vamos a crear una columna que nos indique el tiempo de juego en horas redondeado a 2 decimales
df_users_items['playtime_forever_hours'] = round(df_users_items['playtime_forever'] / 60, 2)

C:\Users\jacev\AppData\Local\Temp\ipykernel_14360\2030160779.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_users_items['playtime_forever_hours'] = round(df_users_items['playtime_forever'] / 60, 2)


In [25]:
# Eliminar la columna 'playtime_forever'
df_users_items.drop(['playtime_forever'], axis=1, inplace=True)

C:\Users\jacev\AppData\Local\Temp\ipykernel_14360\611850048.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_users_items.drop(['playtime_forever'], axis=1, inplace=True)


In [26]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3285246 entries, 0 to 5170013
Data columns (total 4 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   user_id                 object 
 1   item_id                 int32  
 2   item_name               object 
 3   playtime_forever_hours  float64
dtypes: float64(1), int32(1), object(2)
memory usage: 112.8+ MB


In [27]:
df_users_items

,user_id,item_id,item_name,playtime_forever_hours
0,76561197970982479,10,Counter-Strike,0.10
2,76561197970982479,30,Day of Defeat,0.12
8,76561197970982479,300,Day of Defeat: Source,78.88
9,76561197970982479,240,Counter-Strike: Source,30.88
10,76561197970982479,3830,Psychonauts,5.55
...,...,...,...,...
5170007,76561198329548331,304930,Unturned,11.28
5170008,76561198329548331,227940,Heroes & Generals,0.72
5170011,76561198329548331,388490,One Way To Die: Steam Edition,0.05
5170012,76561198329548331,521570,You Have 10 Seconds 2,0.07


In [28]:
df_users_items.describe()

,item_id,playtime_forever_hours
count,3.285246e+06,3.285246e+06
mean,1.772373e+05,2.592093e+01
std,1.266741e+05,1.120172e+02
min,1.000000e+01,2.000000e-02
25%,3.912000e+04,7.300000e-01
50%,2.149500e+05,3.420000e+00
75%,2.630600e+05,1.347000e+01
max,5.286600e+05,1.071288e+04


In [29]:
# Guardar el dataframe en un archivo csv en la carpeta clean
df_users_items.to_csv('../data/clean/users_items.csv.gz', compression='gzip', index=False, encoding='utf-8')

In [30]:
# Guardar el dataframe en un archivo parquet en la carpeta clean
df_users_items.to_parquet('../data/clean/users_items.parquet.gzip', compression='gzip', index=False)